In [ ]:
import numpy as np
import pandas as pd
import ast

In [ ]:
df_movies = pd.read_csv('tmdb_5000_movies.csv')
df_credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
df_movies.head()

In [ ]:
df_credits.head()

In [ ]:
movies = df_movies.merge(df_credits,on='title')

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.loc[movies['overview'].isnull()]


In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
def cleaner(text):
    list = []
    for i in ast.literal_eval(text):
        list.append(i['name'])

    return list

In [ ]:
movies['genres'] = movies['genres'].apply(cleaner)
movies['keywords'] = movies['keywords'].apply(cleaner)

In [ ]:
movies.head()

In [ ]:
def menimizer(text):
    list = []
    counter = 0

    for i in ast.literal_eval(text):
        if counter < 3:
            list.append(i['name'])

            counter += 1
    return list

In [ ]:
movies['cast'] = movies['cast'].apply(menimizer)

In [ ]:
len(movies['cast'][0])

In [ ]:
movies.head()

In [ ]:
def for_crew(text):
    list = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            list.append(i['name'])

    return list

In [ ]:
movies['crew'] = movies['crew'].apply(for_crew)

In [ ]:
movies['crew'][0]

In [ ]:
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [ ]:
def replace(text):
    list = []
    for i in text:
        list.append(i.replace(' ',''))

    return list

In [ ]:
movies['cast'] = movies['cast'].apply(replace)
movies['crew'] = movies['crew'].apply(replace)
movies['genres'] = movies['genres'].apply(replace)
movies['keywords'] = movies['keywords'].apply(replace)

In [ ]:
movies['container'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


In [ ]:
new_data = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [ ]:
new_data['container'] = new_data['container'].apply(lambda x: ' '.join(x))

In [ ]:
new_data.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_data['container']).toarray()

In [ ]:
vector[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
new_data[new_data['title'] == 'Avatar'].index[0]


In [ ]:
def recommend(movie):
    index = new_data[new_data['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_data.iloc[i[0]].title)

In [ ]:
recommend('Avatar')